In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

torch_device = 'cuda'

QA_TOKENIZER = AutoTokenizer.from_pretrained("bert-large-cased-whole-word-masking-finetuned-squad")
QA_MODEL = AutoModelForQuestionAnswering.from_pretrained("bert-large-cased-whole-word-masking-finetuned-squad")

QA_MODEL.to(torch_device)
QA_MODEL.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [2]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [3]:
import sentencepiece
from transformers import BartTokenizer, BartForConditionalGeneration
SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')
SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')

#from transformers import PegasusTokenizer, PegasusForConditionalGeneration
#SUMMARY_TOKENIZER = PegasusTokenizer.from_pretrained('mayu0007/pegasus_large_covid')
#SUMMARY_MODEL = PegasusForConditionalGeneration.from_pretrained('mayu0007/pegasus_large_covid')

SUMMARY_MODEL.to(torch_device)
SUMMARY_MODEL.eval()
def Summarizer(string_all):    
    if 1==1:
        ## try generating an exacutive summary with bart abstractive summarizer
        allAnswersTxt = string_all.replace('\n','')

        answers_input_ids = SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
        #answers_input_ids = SUMMARY_TOKENIZER.prepare_seq2seq_batch([allAnswersTxt], return_tensors='pt', truncation=True, padding='longest')['input_ids'].to(torch_device)
        summary_ids = SUMMARY_MODEL.generate(answers_input_ids,
                                               num_beams=4,
                                               length_penalty=0.8,
                                               repetition_penalty=2.0,
                                               min_length=5,
                                               no_repeat_ngram_size=0,
                                                do_sample=False )

        exec_sum = SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
        return exec_sum

# Program to find most frequent  
# element in a list 
  
from collections import Counter 
  
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0]

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForSequenceClassification
#hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer_nli = AutoTokenizer.from_pretrained(hg_model_hub_name, use_fast=False)
model_nli = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)
model_nli.to(torch_device)
model_nli.eval()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 1024)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=1024, out_features=4096, bias=True)
          (layer_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,),

In [65]:
from pyserini.search import pysearch

import re
import sys
from anglicize import anglicize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from spacy.tokenizer import Tokenizer
from nltk.tokenize import word_tokenize
from spacy.lang.en import English
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
nlp.add_pipe("merge_noun_chunks")
tokenizer = Tokenizer(nlp.vocab)
from timeit import default_timer as timer
import time
import json
import numpy as np
import statistics
import pandas as pd

dataset_path = "dev.jsonl"
# dataset_path = "feverous_test_unlabeled.jsonl"
output_path = dataset_path.replace(".jsonl", "_predictions.jsonl")
evaluation_path = dataset_path.replace(".jsonl", "_evaluation.jsonl")

with open(dataset_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

In [5]:
import sqlalchemy as sqla

db_fullpath = "feverous_wikiv1.db"
db = sqla.create_engine("sqlite:///{}".format(db_fullpath))

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=db)
sql_session = Session()

In [82]:
lucene_dir = 'anserini/indexes/fever/lucene-index-fever-paragraph'
searcher = pysearch.SimpleSearcher(lucene_dir)

def get_relevant_pages(claim, FULL_CLAIM=True, SPACY_ENTITIES=True, CASE_ENTITIES=True, LINKED_PAGES=False, LINKING_PAGES=False, N_RESULTS=70):
    
    if not SPACY_ENTITIES and not CASE_ENTITIES and not LINKED_PAGES:
        FULL_CLAIM = True
    
    start_timer = timer()

    claim = nlp(claim.replace(" ", " ").replace("­", " ")) # Replaces the obnoxious space character with normal space

    keywords = set()
    entities = set()

    if SPACY_ENTITIES:
        spacy_entities = [entity.text for entity in claim.ents if entity.label_ != "DATE"]
        entities.update(spacy_entities)

    if CASE_ENTITIES:
        case_entities = set()
        chunks = claim.noun_chunks
        for chunk in chunks:
            for token in tokenizer(chunk.text):
                if token.text[0].isupper():
                    case_entities.add(chunk.text)
                    break

        entities.update(case_entities)
        #print(case_entities)
        #print(entities)
        #sys.exit(0)

    keywords.update(entities)

    if not FULL_CLAIM:
        search_query = (", ".join(keywords) + '"' + '", "'.join(entities) + '"')
#            search_query = (", ".join(keywords) + ", ".join(entities))
    else:
        search_query = (claim.text + ' "' + '", "'.join(keywords) + '" "' + '", "'.join(entities) + '"')
#            search_query = (claim.text + " " + ", ".join(keywords) + ", ".join(entities))

    if FULL_CLAIM or keywords:
        try:
            lucene_hits = searcher.search(search_query, k=N_RESULTS) #.encode("utf-8")
        except:
            lucene_hits = None
    else:
        lucene_hits = None

    hit_dictionary = {}

    if lucene_hits:

        for hit in lucene_hits:
            hit_abs = hit.lucene_document.get("raw").split(" ~~ ")
            step = 10
            counter = 0
            for i in range(0, len(hit_abs), step):
                temp_abs = " ~~ ".join(hit_abs[i:min([len(hit_abs), i+step])])
                hit_dict = {"abstract": None, "real_abstract": None, "title": None}
#                 hit_dict['abstract']=hit.lucene_document.get("raw")
#                 hit_dict['real_abstract']=hit.lucene_document.get("raw")
                hit_dict['title'] = str(hit.docid)
                hit_dict['abstract'] = temp_abs
                hit_dict['real_abstract'] = temp_abs
                hit_dictionary["{}_{}".format(hit.docid, counter)] = hit_dict
                counter += 1
    
        linked_pages = set()
        if LINKED_PAGES:
            for hit in lucene_hits:
                links = re.findall(r"(?:\[\[)(.*?)(?:\|)", hit.raw)
                for link in links:
                    linked_pages.update([link.replace("_", " ").encode("latin-1", "ignore").decode("utf-8", "ignore")])
                    
        linking_pages = set()
        if LINKING_PAGES and sql_session:
            linking_results = sql_session.execute('select target, sources from inlinks where target IN ("{}")'.format('", "'.join([hit.docid.replace('"', '""') for hit in lucene_hits])))
            for row in linking_results:
                linking_pages.update(row["sources"].split(";"))
                
            for linking_page in linking_pages:
                if linking_page+"_0" not in hit_dictionary:
                    page_raw = sql_session.execute('select * from wiki where id = "{}"'.format(linking_page.replace('"', '""').replace("'", "''"))).fetchone()
                    if not page_raw:
                        continue
                    page_raw = page_raw[1]
                    hit_dict = {"abstract": page_raw, "real_abstract": page_raw, "title": linking_page}
                    hit_dictionary["{}_{}".format(linking_page, "_0")] = hit_dict
                    
                    
        found_pages = [hit.docid for hit in lucene_hits]
        for i in range(0, len(found_pages)):

            try:
                found_pages[i] = found_pages[i].encode("latin-1", "ignore").decode("utf-8", "ignore")
                #print("Done:",found_pages[i])
            except:
                import regex
                print(found_pages[i], anglicize(found_pages[i]), regex.sub(r'[^\p{Latin}]', '', found_pages[i]).encode("latin-1").decode("utf-8"))
                
        found_pages = set(found_pages)

        hit_scores = [hit.score for hit in lucene_hits]
        hit_score_min = min(hit_scores)
        hit_score_25 = np.percentile(hit_scores, 25)
        hit_score_mean = np.mean(hit_scores)
        hit_score_median = statistics.median(hit_scores)
        hit_score_75 = np.percentile(hit_scores, 75)
        hit_score_max = max(hit_scores)
    
    else:
        
        is_found = False
        found_pages = set()
        entities = set()
        keywords = set()
        linked_pages = set()
        linking_pages = set()
        lucene_hits = []
        hit_scores = []
        hit_score_min = None
        hit_score_25 = None
        hit_score_mean = None
        hit_score_median = None
        hit_score_75 = None
        hit_score_max = None

    end_timer = timer()
    elapsed = int(end_timer - start_timer)
    elapsed_formatted = time.strftime("%H:%M:%S", time.gmtime(elapsed))
        
    result = [claim.text, elapsed_formatted, 
                found_pages, 
                keywords, 
                linked_pages, 
                linking_pages,
                len(lucene_hits), 
                hit_score_min, hit_score_25, hit_score_mean, hit_score_median, 
                hit_score_75, hit_score_max, hit_scores]

    #print(result)
    
    result = pd.DataFrame([result], columns=["CLAIM", "ELAPSED", 
                                             "PAGES_FOUND",
                                             "KEYWORDS", 
                                             "LINKED_PAGES",
                                             "LINKING_PAGES",
                                             "N_LUCENE_HITS", 
                                             "HIT_SCORE_MIN", "HIT_SCORE_25", "HIT_SCORE_MEAN", "HIT_SCORE_MEDIAN", 
                                             "HIT_SCORE_75", "HIT_SCORE_MAX", "HIT_SCORES"])
    
    return result, hit_dictionary

In [7]:
def CheckEntailmentNeturalorContradict(ranked_aswers, string_all, pandasData):
    if 1 == 1:
        premise_list = []
        hypothesis_list = []
        entailment_prob = []
        neutral_prob = []
        contradict_prob = []
        final_result_list = []
        answers_to_check = 3
        if len(ranked_aswers) > answers_to_check:
            v_range = answers_to_check
        else:
            v_range = len(ranked_aswers)
        for i in range(v_range):
            if pandasData[i][2] >= 0.6:
                max_length = 512
#                 candidate_answer = Summarizer(ranked_aswers[i])
                candidate_answer = ranked_aswers[i]
                # premise = "symptoms of covid-19 range from none (asymptomatic) to severe pneumonia and it can be fatal. fever and cough are the most common symptoms in patients with 2019-ncov infection. Several people have muscle soreness or fatigue as well as ards. diarrhea, hemoptysis, headache, sore throat, shock, and other symptoms only occur in a small number of patients"
                if len(candidate_answer) > len(ranked_aswers[i]):
                    premise = ranked_aswers[i]
                else:
                    premise = candidate_answer
                # premise = "covid-19 symptom is highly various in each patient, with fever, fatigue, shortness of breath, and cough as the main presenting symptoms. patient with covid-19 may shows severe symptom with severe pneumonia and ards, mild symptom resembling simple upper respiration tract infection, or even completely asymptomatic. approximately 80 % of cases is mild "
                hypothesis = string_all
                tokenized_input_seq_pair = tokenizer_nli.encode_plus(
                    premise,
                    hypothesis,
                    max_length=max_length,
                    return_token_type_ids=True,
                    truncation=True,
                )

                input_ids = (
                    torch.Tensor(tokenized_input_seq_pair["input_ids"])
                    .long()
                    .unsqueeze(0)
                    .to(torch_device)
                )
                # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
                token_type_ids = (
                    torch.Tensor(tokenized_input_seq_pair["token_type_ids"])
                    .long()
                    .unsqueeze(0)
                    .to(torch_device)
                )
                attention_mask = (
                    torch.Tensor(tokenized_input_seq_pair["attention_mask"])
                    .long()
                    .unsqueeze(0)
                    .to(torch_device)
                )

                outputs = model_nli(
                    input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    labels=None,
                )
                # Note:
                # "id2label": {
                #     "0": "entailment",
                #     "1": "neutral",
                #     "2": "contradiction"
                # },

                predicted_probability = torch.softmax(outputs[0], dim=1)[
                    0
                ].tolist()  # batch_size only one

                # print("Premise:", premise)
                premise_list.append(premise)
                # print("Hypothesis:", hypothesis)
                hypothesis_list.append(hypothesis)
                # print("input Ans:",ranked_aswers[i])
                # print("Entailment:", predicted_probability[0])
                entailment_prob.append(predicted_probability[0])
                # print("Neutral:", predicted_probability[1])
                neutral_prob.append(predicted_probability[1])
                # print("Contradiction:", predicted_probability[2])
                contradict_prob.append(predicted_probability[2])
        if len(premise_list) > 0:
            avg_entailment = np.average(entailment_prob)
            avg_neutral = np.average(neutral_prob)
            avg_contradiction = np.average(contradict_prob)
            zipped_list = zip(entailment_prob, neutral_prob, contradict_prob)
            for key, val in enumerate(hypothesis_list):
                if neutral_prob[key] > 0.8:
                    final_result = "Neutral"
                elif entailment_prob[key] > contradict_prob[key]:
                    final_result = "True"
                    final_result_list.append(final_result)
                else:
                    final_result = "False"
                    final_result_list.append(final_result)
            if len(final_result_list) > 0:
                verdict = most_frequent(final_result_list)
            else:
                verdict = "Neutral"
        else:
            #       zipped_list = zip(0,1,0)
            zipped_list = zip([0], [1], [0])
            verdict = "Neutral"
    return zipped_list, verdict

In [8]:
def get_canonical_title(title):
    return title.split("_")[0]

def get_entities(text):
    claim = nlp(text.replace(" ", " ").replace("­", " ")) # Replaces the obnoxious space character with normal space

    entities = set()
    
    spacy_entities = [entity.text for entity in claim.ents if entity.label_ != "DATE"]
    entities.update(spacy_entities)
    
    claim_processed = nlp(text.replace(" ", " ").replace("­", " ").replace("'", ". "))
    
    spacy_entities_processed = [entity.text for entity in claim_processed.ents if entity.label_ != "DATE"]
    entities.update(spacy_entities)

    case_entities = set()
    chunks = claim.noun_chunks
    for chunk in chunks:
        for token in tokenizer(chunk.text):
            if token.text[0].isupper():
                case_entities.add(chunk.text)
                break

        entities.update(case_entities)
    
    return entities

def get_people(text):
    
    claim = nlp(text.replace(" ", " ").replace("­", " ")) # Replaces the obnoxious space character with normal space
    
    people = set()

    for ent in claim.ents:
        if ent.label_ == "PERSON":
            people.add(ent.text)
            
    claim_processed = nlp(text.replace(" ", " ").replace("­", " ").replace("'", ". "))
    
    for ent in claim_processed.ents:
        if ent.label_ == "PERSON":
            people.add(ent.text)
            
    case_entities = set()
    chunks = claim_processed.noun_chunks
    for chunk in chunks:
        for token in tokenizer(chunk.text):
            if token.text[0].isupper():
                case_entities.add(chunk.text)
                break
                
#     print(case_entities)
                
    for entity in case_entities:
        candidate_entities = nlp(entity).ents
        for candidate in candidate_entities:
            if candidate.label_ == "PERSON":
                people.add(candidate.text)
            
    return people

In [9]:
def get_relevant_cells(page, search_for):
    import json
    import Levenshtein

    result = sql_session.execute('''select * from wiki where id = "{}"'''.format(page.replace('"', '""').replace("'", "''"))).fetchone()
    if not result:
#         print("Page not found:",page)
        return []
    data = json.loads(result[1])
    tables = [element for element in data["order"] if "table_" in element]

    matches = []
    for table in tables:
        rows = data[table]["table"]
        for row in rows:
            for cell in row:
                for searching in search_for:
                    similarities = [
                        Levenshtein.ratio(searching, cell["value"]), 
                        Levenshtein.ratio(searching, re.sub(r"(\[\[).*?(\|)|]]", "", cell["value"], flags=re.MULTILINE))
                    ]

                    if max(similarities) >= 0.8:
                        matches.append([max(similarities), cell["id"]])
                        break

    matches.sort(key=lambda x: float(x[0]), reverse=True)

    return [match[1] for match in matches]

In [75]:
import json
import ast
import pandas as pd
import numpy
import sys
import os

# print(os.environ.get("MKL_THREADING_LAYER"))
# os.environ["MKL_THREADING_LAYER"] = "None"

# print(os.environ.get("BUILD_DOCS"))
# os.environ["BUILD_DOCS"] = "1"

# !pip install --upgrade --force-reinstall torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.1+cu102.html
    
import torch_scatter

# !pip install transformers
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering, TapasConfig

table_tokenizer = AutoTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")

# config = TapasConfig(drop_rows_to_fit=True)

table_model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")

def get_table_evidence(page, claim):
    from transformers import TapasTokenizer, TapasForQuestionAnswering
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    import re
    result = sql_session.execute("select * from wiki where id = '{}'".format(page.replace('"', '""').replace("'", "''"))).fetchone()
    if not result:
        return []
    data = json.loads(result[1])
    tables = [element for element in data["order"] if "table_" in element]
    if not tables:
        return []
    cell_ids = set()
    queries = [claim]
    
    for table in tables:
#         print(page, table)
        table_data = json_table_to_dict(data[table]["table"], crop=True)
        if not table_data:
            continue
        table_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in table_data.items() ])).replace(np.nan,' ')
        inputs = table_tokenizer(table=table_df, queries=queries, padding='max_length', return_tensors="pt")
        if len(inputs["input_ids"].view(-1)) > 1000:
            continue
        outputs = table_model(**inputs)
        predicted_answer_coordinates, predicted_aggregation_indices = table_tokenizer.convert_logits_to_predictions(
                inputs,
                outputs.logits.detach(),
                outputs.logits_aggregation.detach(),
                cell_classification_threshold=0.3
        )
        # let's print out the results:
        id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3:"COUNT"}
        aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]
        answers = []
        for coordinates in predicted_answer_coordinates:
            if len(coordinates) == 1:
                # only a single cell:
                answers.append(table_df.iat[coordinates[0]])
            else:
                # multiple cells
                cell_values = []
                for coordinate in coordinates:
                    cell_values.append(table_df.iat[coordinate])
                    
                answers.extend(cell_values)
                
        for answer in answers:
            for row in data[table]["table"]:
                row_relevant = False
                for cell in row:
                    if answer == re.sub(r"(\[\[).*?(\|)|]]", "", cell["value"], flags=re.MULTILINE) or answer == cell["value"]:
                        row_relevant = True
                        cell_ids.add(cell["id"])
                        break
                        
                if row_relevant:
                    for cell in row:
#                         print(cell["id"], cell["value"])
                        cell_ids.add(cell["id"])
                        
    return list(cell_ids)

In [71]:
from itertools import product

def table_to_2d(table_tag):
    rowspans = []  # track pending rowspans
    rows = table_tag.find_all('tr')

    # first scan, see how many columns we need
    colcount = 0
    for r, row in enumerate(rows):
        cells = row.find_all(['td', 'th'], recursive=False)
        # count columns (including spanned).
        # add active rowspans from preceding rows
        # we *ignore* the colspan value on the last cell, to prevent
        # creating 'phantom' columns with no actual cells, only extended
        # colspans. This is achieved by hardcoding the last cell width as 1. 
        # a colspan of 0 means “fill until the end” but can really only apply
        # to the last cell; ignore it elsewhere. 
        colcount = max(
            colcount,
            sum(int(c.get('colspan', 1)) or 1 for c in cells[:-1]) + len(cells[-1:]) + len(rowspans))
        # update rowspan bookkeeping; 0 is a span to the bottom. 
        rowspans += [int(c.get('rowspan', 1)) or len(rows) - r for c in cells]
        rowspans = [s - 1 for s in rowspans if s > 1]

    # it doesn't matter if there are still rowspan numbers 'active'; no extra
    # rows to show in the table means the larger than 1 rowspan numbers in the
    # last table row are ignored.

    # build an empty matrix for all possible cells
    table = [[None] * colcount for row in rows]

    # fill matrix from row data
    rowspans = {}  # track pending rowspans, column number mapping to count
    for row, row_elem in enumerate(rows):
        span_offset = 0  # how many columns are skipped due to row and colspans 
        for col, cell in enumerate(row_elem.find_all(['td', 'th'], recursive=False)):
            # adjust for preceding row and colspans
            col += span_offset
            while rowspans.get(col, 0):
                span_offset += 1
                col += 1

            # fill table data
            rowspan = rowspans[col] = int(cell.get('rowspan', 1)) or len(rows) - row
            colspan = int(cell.get('colspan', 1)) or colcount - col
            # next column is offset by the colspan
            span_offset += colspan - 1
            value = cell.get_text()
            for drow, dcol in product(range(rowspan), range(colspan)):
                try:
                    table[row + drow][col + dcol] = value
                    rowspans[col + dcol] = rowspan
                except IndexError:
                    # rowspan or colspan outside the confines of the table
                    pass

        # update rowspan bookkeeping
        rowspans = {c: s - 1 for c, s in rowspans.items() if s > 1}

    return table

def get_html_table(table, remove_links):
    html_table = "<table>"
    for row in table:
        html_table += "<tr>"
        for cell in row:
            if remove_links:
                html_table += '<td rowspan="{}" colspan="{}">{}</td>'.format(cell["row_span"], cell["column_span"], re.sub(r"(\[\[).*?(\|)|]]", "", cell["value"], flags=re.MULTILINE)) 
            else:
                html_table += '<td rowspan="{}" colspan="{}">{}</td>'.format(cell["row_span"], cell["column_span"], cell["value"])
        html_table += "</tr>"
                
    html_table += "</table>"
    return html_table

# json.loads(sql_session.execute("select * from wiki where id = 'John Laurie'").fetchone()[1])["table_1"]["table"]

def json_table_to_dict(table, remove_links=True, crop=False):
    from bs4 import BeautifulSoup
    if "table" in table:
        table = table["table"]
    list_table = table_to_2d(BeautifulSoup(get_html_table(table, remove_links=remove_links)))
    if not list_table:
        return {}
    else:
#         print(list_table)
        pass
    
    header_index = None
    for row_id, row in enumerate(table):
        if row and row[0]["is_header"] == True:
            header_index = row_id
            break
            
    list_table = list_table[header_index:]
    
#     if table[0][0]["is_header"] != True:
#         return {}

#         print("Tablo önce:",list_table)
#         max_n_col = 0
#         for row in table:
#             if len(row) > max_n_col:
#                 max_n_col = len(row)
#         header = [list(range(0, max_n_col))]
#         print(header)
#         list_table = header.extend(list_table)
#         print("Tablo sonra:",list_table)
        
    if crop:
        table_cropped = []
#         print(list_table)
        for row in list_table:
#             print("Row:",row)
            if len(" ".join(row)) > 700:
                continue
            else:
                table_cropped.append(row)
    else:
        table_cropped = list_table
        
    dict_table = {}
    if not table_cropped:
        return {}
    for col_id, column in enumerate(table_cropped[0]):
        dict_table[column] = [row[col_id] for row in table_cropped[1:]]
    return dict_table

def get_numpy_table(table, remove_links=True):
    if "table" in table:
        table = table["table"]
    num_col = len(table[0])
    num_rows = len(table)
    data_x =np.empty(shape=(num_rows,num_col), dtype=object )

    for elements in table:
        for element in elements:
            id = element["id"].split("_")
            for j in range(int(element["column_span"])):
                for i in range(int(element["row_span"])):
                    k=0
                    while (data_x[int(id[-2])+i][int(id[-1])+j+k] !=None):
                        if ((int(id[-1])+j+k+1)<num_col):
                            k = k+1
                        else:
                            break

                    if remove_links:
                        data_x[int(id[-2])+i][int(id[-1])+j+k] = re.sub(r"(\[\[).*?(\|)|]]", "", element["value"], flags=re.MULTILINE)
                    else:
                        data_x[int(id[-2])+i][int(id[-1])+j+k] = element["value"]                                                  
                    # print(element["value"])

        # print(element["value"])
    return data_x

In [45]:
problems = [212+3500,
275+3500,
276+3500,
430+3500,
712+3500,
1393+3500,
1652+3500,
1659+3500,
1718+3500,
1747+3500,
1830+3500,
1836+3500,
1957+3500,
2074+3500,
2189+3500,
2231+3500,
2256+3500,
2273+3500,
2302+3500]

problems

[3712,
 3775,
 3776,
 3930,
 4212,
 4893,
 5152,
 5159,
 5218,
 5247,
 5330,
 5336,
 5457,
 5574,
 5689,
 5731,
 5756,
 5773,
 5802]

In [92]:
import Levenshtein

def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.compat.v1.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.compat.v1.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT('/home/titanx/kaggle/working/sentence_wise_email/module/module_useT')

def get_answers(query, hit_dictionary, improved_name_check=False, include_previous_sentence=True, display_table=False):
    import re
    for idx,v in hit_dictionary.items():
        abs_dirty = v['abstract']
        real_abs_dirty = v['real_abstract']
        #abs_dirty = v['paragraph']
        # looks like the abstract value can be an empty list
        v['abstract_paragraphs'] = []
        v['abstract_full'] = ''
        v['real_abstract_full'] = ''
        v['real_abstract_paragraphs']=[]

        if abs_dirty:
            # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
            # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
            # and a new entry that is full abstract text as both could be valuable for BERT derrived QA

            if isinstance(abs_dirty, list):
                for p in abs_dirty:
                    v['abstract_paragraphs'].append(p['text'])
                    v['abstract_full'] += p['text'] + ' \n\n'

            # looks like in some cases the abstract can be straight up text so we can actually leave that alone
            if isinstance(abs_dirty, str):
                v['abstract_paragraphs'].append(abs_dirty)
                v['abstract_full'] += abs_dirty + ' \n\n'
        if real_abs_dirty:
            # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
            # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
            # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


            if isinstance(real_abs_dirty, list):
                for p in real_abs_dirty:
                    v['real_abstract_paragraphs'].append(p['text'])
                    v['real_abstract_full'] += p['text'] + ' \n\n'

            # looks like in some cases the abstract can be straight up text so we can actually leave that alone
            if isinstance(abs_dirty, str):
                v['real_abstract_paragraphs'].append(real_abs_dirty)
                v['real_abstract_full'] += real_abs_dirty + ' \n\n'
        v["indexed_para"]=v["abstract_full"][len(v["real_abstract_full"]):]
        
        
        v["abstract_full_processed"] = re.sub(r"(\[\[).*?(\|)|]]", "", v["abstract_full"], flags=re.MULTILINE) # Link format removal
        v["abstract_full_processed"] = re.sub(r"((sentence|table|cell|section|list|item)(_[0-9]+)+ )", "", v["abstract_full_processed"], flags=re.MULTILINE)
        v["abstract_full_processed"] = v["abstract_full_processed"].replace("~~", ". ~~ .")
#     v["abstract_full_processed"] = v["abstract_full_processed"].replace("~~", " ~~ ")
        

#     def embed_useT(module):
#         with tf.Graph().as_default():
#             sentences = tf.compat.v1.placeholder(tf.string)
#             embed = hub.Module(module)
#             embeddings = embed(sentences)
#             session = tf.compat.v1.train.MonitoredSession()
#         return lambda x: session.run(embeddings, {sentences: x})
#     embed_fn = embed_useT('/home/titanx/kaggle/working/sentence_wise_email/module/module_useT')

    import numpy as np
    #Reconstruct text for generating the answer text from the BERT result, do the necessary text preprocessing and construct the answer.
    def reconstructText(tokens, start=0, stop=-1):
        tokens = tokens[start: stop]
        if '[SEP]' in tokens:
            sepind = tokens.index('[SEP]')
            tokens = tokens[sepind+1:]
        txt = ' '.join(tokens)
        txt = txt.replace(' ##', '')
        txt = txt.replace('##', '')
        txt = txt.strip()
        txt = " ".join(txt.split())
        txt = txt.replace(' .', '.')
        txt = txt.replace('( ', '(')
        txt = txt.replace(' )', ')')
        txt = txt.replace(' - ', '-')
        txt_list = txt.split(' , ')
        txt = ''
        nTxtL = len(txt_list)
        if nTxtL == 1:
            return txt_list[0]
        newList =[]
        for i,t in enumerate(txt_list):
            if i < nTxtL -1:
                if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                    newList += [t,',']
                else:
                    newList += [t, ', ']
            else:
                newList += [t]
        return ''.join(newList)


    def makeBERTSQuADPrediction(title, document, question):
        title = get_canonical_title(title)
        ## we need to rewrite this function so that it chuncks the document into 250-300 word segments with
        ## 50 word overlaps on either end so that it can understand and check longer abstracts
        ## Get chuck of documents in order to overcome the BERT's max 512 token problem. First count the number of tokens. For example if it is
        ##   larger than 2048 token determine the split size and overlapping token count. Split the document into pieces to fed into BERT encoding. Overlapping
        ##      is necessary to maintain the coherence between the splits. If the token count is larger than some number 2560 token approximately this will fail.
        nWords = len(document.split())
        input_ids_all = QA_TOKENIZER.encode(question, document)
        tokens_all = QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
        overlapFac = 1.1
        if len(input_ids_all)*overlapFac > 2560:
            nSearchWords = int(np.ceil(nWords/6))
            fifth = int(np.ceil(nWords/5))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[fifth-int(nSearchWords*overlapFac/2):fifth+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*2-int(nSearchWords*overlapFac/2):fifth*2+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*3-int(nSearchWords*overlapFac/2):fifth*3+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*4-int(nSearchWords*overlapFac/2):fifth*4+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]

        elif len(input_ids_all)*overlapFac > 2048:
            nSearchWords = int(np.ceil(nWords/5))
            quarter = int(np.ceil(nWords/4))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
            
        elif len(input_ids_all)*overlapFac > 1536:
            nSearchWords = int(np.ceil(nWords/4))
            third = int(np.ceil(nWords/3))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
            
        elif len(input_ids_all)*overlapFac > 1024:
            nSearchWords = int(np.ceil(nWords/3))
            middle = int(np.ceil(nWords/2))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        elif len(input_ids_all)*overlapFac > 512:
            nSearchWords = int(np.ceil(nWords/2))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        else:
            input_ids = [input_ids_all]
        absTooLong = False    
        
        answers = []
        cons = []
        #print(input_ids)
        for iptIds in input_ids:
            tokens = QA_TOKENIZER.convert_ids_to_tokens(iptIds)
            #print(tokens)
            sep_index = iptIds.index(QA_TOKENIZER.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(iptIds) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b
            assert len(segment_ids) == len(iptIds)
            n_ids = len(segment_ids)
            #print(n_ids)
            if n_ids < 512:
                outputs = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                                        token_type_ids=torch.tensor([segment_ids]).to(torch_device))
                #print(QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                #                        token_type_ids=torch.tensor([segment_ids]).to(torch_device)))
            else:
                #this cuts off the text if its more than 512 words so it fits in model space
                #need run multiple inferences for longer text. add to the todo. I will try to expand this to longer sequences if it is necessary. 
#                 print('****** warning only considering first 512 tokens, document is '+str(nWords)+' words long.  There are '+str(n_ids)+ ' tokens')
                absTooLong = True
                outputs= QA_MODEL(torch.tensor([iptIds[:512]]).to(torch_device), 
                                        token_type_ids=torch.tensor([segment_ids[:512]]).to(torch_device))
            start_scores=outputs.start_logits
            end_scores=outputs.end_logits
            start_scores = start_scores[:,1:-1]
            end_scores = end_scores[:,1:-1]
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)
            #print(answer_start, answer_end)
            answer = reconstructText(tokens, answer_start, answer_end+2)
        
            if answer.startswith('. ') or answer.startswith(', '):
                answer = answer[2:]
                
            c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
            answers.append(answer)
            cons.append(c)
        
        maxC = max(cons)
        iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
        
        entities = get_entities(question)
        people = get_people(question)
        
        title = get_canonical_title(title)
        
#         entity_similarities = [Levenshtein.ratio(entity, title) for entity in entities]
#         person_similarities = [Levenshtein.ratio(person, title) for person in people]
        
        person_found = False
        
        for person in people:
#             print(person, title)
            if person in title or title in person:
#                 print("buldu")
                person_found = True
                break
        
#         if person_similarities and max(person_similarities) >= 0.8:
#         if title in people:
        if person_found:
#             confidence = max([0.95, cons[iMaxC]])
            confidence = 2*cons[iMaxC]
            
#             print(confidence)
#         elif entity_similarities and max(entity_similarities) >= 0.8:
#             confidence = max([0.9, cons[iMaxC]])
        else:
            confidence = cons[iMaxC]
            
        answer = answers[iMaxC]
        
        sep_index = tokens_all.index('[SEP]')
        full_txt_tokens = tokens_all[sep_index+1:]
        
        abs_returned = reconstructText(full_txt_tokens)

        ans={}
        ans['answer'] = answer
        #print(answer)
        if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
            ans['confidence'] = -1000000
        else:
            #confidence = torch.max(start_scores) + torch.max(end_scores)
            #confidence = np.log(confidence.item())
            ans['confidence'] = confidence
        #ans['start'] = answer_start.item()
        #ans['end'] = answer_end.item()
        ans['abstract_bert'] = abs_returned
        ans['abs_too_long'] = absTooLong
        return ans

    from tqdm import tqdm
    def searchAbstracts(hit_dictionary, question):
        abstractResults = {}
#         for k,v in tqdm(hit_dictionary.items()):
        for k,v in hit_dictionary.items():
#             abstract = v['abstract_full']
            abstract = v['abstract_full_processed']
#             print(abstract)
            indexed_para=v['indexed_para']
            title = v["title"]
            if abstract:
                ans = makeBERTSQuADPrediction(title, abstract, question)
                if ans['answer']:
                    confidence = ans['confidence']
#                     print(title,"-",confidence)
                    abstractResults[confidence]={}
                    abstractResults[confidence]['answer'] = ans['answer']
                    #abstractResults[confidence]['start'] = ans['start']
                    #abstractResults[confidence]['end'] = ans['end']
                    abstractResults[confidence]['abstract_bert'] = ans['abstract_bert']
                    abstractResults[confidence]['idx'] = k
                    abstractResults[confidence]['abs_too_long'] = ans['abs_too_long']

                    
        cList = list(abstractResults.keys())
        if cList:
            maxScore = max(cList)
            total = 0.0
            exp_scores = []
            for c in cList:
                s = np.exp(c-maxScore)
                exp_scores.append(s)
            total = sum(exp_scores)
            for i,c in enumerate(cList):
                abstractResults[exp_scores[i]/total] = abstractResults.pop(c)
                
        return abstractResults

    answers = searchAbstracts(hit_dictionary, query)

    workingPath = '/root/kaggle/working'
    import pandas as pd
    import re
    from IPython.core.display import display, HTML

    #from summarizer import Summarizer
    #summarizerModel = Summarizer()
    def displayResults(hit_dictionary, answers, question, improved_name_check=False, include_previous_sentence=True, display_table=False):
        
        question_HTML = '<div style="font-family: Times New Roman; font-size: 28px; padding-bottom:28px"><b>Query</b>: '+question+'</div>'
        #all_HTML_txt = question_HTML
        confidence = list(answers.keys())
        confidence.sort(reverse=True)
        
        page_answers = dict()

        for c in confidence:
#             if c>0 and c <= 1 and len(answers[c]['answer']) != 0:
            if c>0 and len(answers[c]['answer']) != 0:
                if 'idx' not in  answers[c]:
                    continue
                rowData = []
                idx = answers[c]['idx']
                title = hit_dictionary[idx]['title']
                
                full_abs = answers[c]['abstract_bert']
                bert_ans = answers[c]['answer']
                
                
                #split_abs = full_abs.split(bert_ans)
                #x=(split_abs[0][:split_abs[0].rfind('. ')+1])
                #sentance_beginning = x[x.rfind('. ')+1:] + split_abs[0][split_abs[0].rfind('. ')+1:]
                #sentance_beginning = split_abs[0][split_abs[0].rfind('.')+1:]
                x=''
                y=''
                z=''
                t=''

                # print(bert_ans)
                split_abs = full_abs.split(bert_ans)
                
                if split_abs[0][-5:] == "~ ~. ":
                    answer_sentence_start = True
                else:
                    answer_sentence_start = False
                
#                 if get_canonical_title(title) == "Abraham Annan":
#                     print("Buraya bak:",split_abs[0])
#                     print(split_abs[0][-5:])
                    
                if include_previous_sentence and len(split_abs[0].split("~ ~")) > 1:
                    sentance_beginning = " ~ ~ ".join(split_abs[0].split("~ ~")[-2:]) # Also includes the previous sentence
                else:
                    sentance_beginning = split_abs[0].split("~ ~")[-1] # Only the sentence that includes the answer
                if len(split_abs) > 1:
                    sentance_end = split_abs[1].split("~ ~")[0]
                else:
                    sentance_end = ""
                
                if answer_sentence_start:
                    sentance_full = sentance_beginning + "~ ~" + bert_ans + sentance_end
                else:
                    sentance_full = sentance_beginning + bert_ans + sentance_end
                    
#                 print(sentance_full)
                
                sentences = sentance_full.split("~ ~")
                
                abstract_full = hit_dictionary[idx]["abstract_full"]
                
                all_answer_ids = set()
                
                for sentence in sentences:
                
                    for element in abstract_full.split(" ~~ "):
                        element = re.sub(r"(\[\[).*?(\|)|]]", "", element, flags=re.MULTILINE) # Link format removal
                        answer_ids = re.findall(r"((sentence|table|cell|section|list|item)(_[0-9]+)+ )", element)

                        if not answer_ids:
                            continue

                        answer_ids_formatted = set()

                        for ids in answer_ids:
                            if isinstance(ids, str):
                                candidate = ids.replace(" ", "")
                                if "_" in candidate and candidate[0] != "_" and candidate not in ["sentence", "table", "cell", "section", "list", "item"]:  # very hacky
                                    answer_ids_formatted.add(candidate)
                            else:
                                for id in ids:
                                    candidate = id.replace(" ", "")
                                    if "_" in candidate and candidate[0] != "_" and candidate not in ["sentence", "table", "cell", "section", "list", "item"]:  # very hacky
                                        answer_ids_formatted.add(candidate)
                                        
                                        
#                         if get_canonical_title(title) == "Abraham Annan":
#                             print("Abraham Annan",answer_ids, answer_ids_formatted)
                                        
#                         all_answer_ids.extend(answer_ids_formatted)
#                         print(answer_ids_formatted, title)
                        for answer_id_formatted in answer_ids_formatted:
#                             if get_canonical_title(title) == "Abraham Annan":
#                                 print(element)
#                                 print(answer_id_formatted)
#                                 print(element.replace(answer_id_formatted, "").strip())
#                                 print(sentence)
#                                 print("=======")
                            if Levenshtein.ratio(element.replace(answer_id_formatted, "").strip(), sentence) >= 0.9:
                                all_answer_ids.add(answer_id_formatted)
                
                
#                 sentence_pre = abstract_full.split(sentance_full)[0].split("~~")[-1]
                
                
#                 print(sentance_full, full_abs)

#                 answer_ids = re.findall(r"((sentence|table|cell|section|list|item)( _ [0-9]+)+ )", sentance_full)
#                 answer_ids = re.findall(r"((sentence|table|cell|section|list|item)(_[0-9]+)+ )", sentence_pre)
#                 answer_ids_formatted = []

#                 for ids in answer_ids:
#                     if isinstance(ids, str):
#                         candidate = ids.replace(" ", "")
#                         if "_" in candidate and candidate[0] != "_" and candidate not in ["sentence", "table", "cell", "section", "list", "item"]:  # very hacky
#                             answer_ids_formatted.append(candidate)
#                     else:
#                         for id in ids:
#                             candidate = id.replace(" ", "")
#                             if "_" in candidate and candidate[0] != "_" and candidate not in ["sentence", "table", "cell", "section", "list", "item"]:  # very hacky
#                                 answer_ids_formatted.append(candidate)
                                
#                 print(answer_ids_formatted)
                
#                 page_answers[title] = answer_ids_formatted
                page_answers[get_canonical_title(title)] = list(all_answer_ids)
#                 page_answers[title] = element_id

                answers[c]['full_answer'] = sentance_beginning+bert_ans+sentance_end
                answers[c]['partial_answer'] = bert_ans+sentance_end
                answers[c]['sentence_beginning'] = sentance_beginning
                answers[c]['sentence_end'] = sentance_end
                answers[c]['title'] = get_canonical_title(title)
            else:
                answers.pop(c)
        
        
        ## now rerank based on semantic similarity of the answers to the question
        ## Universal sentence encoder
        cList = list(answers.keys())
        allAnswers = [answers[c]['full_answer'] for c in cList]
        
        messages = [question]+allAnswers
        
        encoding_matrix = embed_fn(messages)
        similarity_matrix = np.inner(encoding_matrix, encoding_matrix)
        rankings = similarity_matrix[1:,0]
        
        for i,c in enumerate(cList):
            answers[rankings[i]] = answers.pop(c)

        ## now form pandas dv
        confidence = list(answers.keys())
        confidence.sort(reverse=True)
        pandasData = []
        ranked_aswers = []
        full_abs_list=[]
        for c in confidence:
            rowData=[]
            title = answers[c]['title']
            idx = answers[c]['idx']
            rowData += [idx]            
            sentance_html = '<div>' +answers[c]['sentence_beginning'] + " <font color='red'>"+answers[c]['answer']+"</font> "+answers[c]['sentence_end']+'</div>'
            answer_key = answers[c]['answer'].split("\t")[-1].strip() if "\t" in answers[c]['answer'] else answers[c]['answer'].strip()

            # rowData += [sentance_html, answer_key, c,title]
            rowData += [sentance_html, c,title]
            pandasData.append(rowData)
            ranked_aswers.append(' '.join([answers[c]['full_answer']]))
            full_abs_list.append(' '.join([answers[c]['abstract_bert']]))
        
        people = get_people(question)
        
        for row_id, row in enumerate(pandasData):
            title = row[3]
            for person in people:
                if improved_name_check and (title in person or person in title or Levenshtein.ratio(title.split(" (")[0], person) >= 0.8):
                    pandasData[row_id][2] = pandasData[row_id][2] * 2
                    break
                elif title in person or person in title:
                    pandasData[row_id][2] = pandasData[row_id][2] * 2
                    break
                    
        pandasData.sort(key=lambda x: float(x[2]), reverse=True)
        
        pdata2 = pandasData
            
        if display_table:
            display(HTML(question_HTML))

        #    display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>Body of Evidence:</b></div>'))

            # df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Answer Key', 'Confidence', 'Title/Link'])
            df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])

            display(HTML(df.to_html(render_links=True, escape=False)))
        return page_answers,full_abs_list,ranked_aswers,pandasData

    return displayResults(hit_dictionary, answers, query, improved_name_check, include_previous_sentence, display_table)




INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
import os

# depth or breadth
# EVIDENCE_RETRIEVAL = "breadth"
EVIDENCE_RETRIEVAL = "depth"

EVAL = True

# with open(output_path, 'w', encoding='utf-8') as f:
#     f.write('{"id": "", "predicted_label": "", "predicted_evidence": ""}\n')

# if os.path.isfile(output_path):
#     os.remove(output_path)
    
# if EVAL and os.path.isfile(evaluation_path):
#     os.remove(evaluation_path)
    
from tqdm import tqdm
for data in tqdm(dataset[101:]):
    
    claim = data["claim"]
    
    if not claim:
        continue
        
#     print(claim)
    
    hit_stats, hit_dictionary = get_relevant_pages(claim, N_RESULTS=10)
    
#     hit_dictionary
#     break
    
    page_answers, full_abs_list, ranked_aswers, pandasData = get_answers(claim, hit_dictionary, improved_name_check=True, include_previous_sentence=True, display_table=False)
#     print(page_answers)

#     print(pandasData)
#     continue

    entities = get_entities(claim)
    people = get_people(claim)

    max_cell_evidence = 25
    
    cell_evidences = []
    for row in pandasData:
        if row[2] < 0.5:
            continue
            
        page = row[3]
        if len(cell_evidences) >= max_cell_evidence:
            break
            
        cells = get_relevant_cells(page, [entity for entity in entities if entity not in people])
        
        try:
            model_cells = get_table_evidence(page, claim)
        except:
            model_cells = []
        
        
#         if not cells and not model_cells:
#             continue
            
#         if not cells:
#             continue
        
        for cell in model_cells:
            if cell not in cells:
                cells.append(cell)
        
        cells = cells[0:min([len(cells),(max_cell_evidence-len(cell_evidences))])]
        
        
        for cell in cells:
            cell_evidences.append([page, cell.split("_", 1)[0], cell.split("_", 1)[1]])
        
#         cell_evidences.extend(cells[0:min([len(cells),(max_cell_evidence-len(cell_evidences))])])
        
    cell_evidences = cell_evidences[0:max_cell_evidence] # Just to be sure
#     print(cell_evidences)
    
    
    
#     print(cell_evidences)

    evidence_probs, verdict = CheckEntailmentNeturalorContradict(ranked_aswers,claim,pandasData)

    if verdict == "False":
        predicted_label = "REFUTES"
    elif verdict == "True":
        predicted_label = "SUPPORTS"
    else:
        predicted_label = "NOT ENOUGH INFO"
        
#     entities = get_entities(claim)
    people = get_people(claim)
    
    for row_id, row in enumerate(pandasData):
        title = row[3]
        for person in people:
            if title in person or person in title:
                pandasData[row_id][2] = pandasData[row_id][2] * 2
                break
    
    pages_by_confidence = [row[0] for row in pandasData]
    
    
#     print(pandasData[0][2])
#     print(pages_by_confidence)
#     print(page_answers["Abraham Annan"])
    
    evidences = []
    
    if EVIDENCE_RETRIEVAL == "depth":
        for page in pages_by_confidence:
            page = get_canonical_title(page)
            idx = page_answers[page]
            for element in idx:
                if len(evidences) < 5:
                    element_type, element_id = element.split("_", 1)
                    evidence = [page, element_type, element_id]
#                     if len(evidences) == 0:
#                         print(evidence)
                    if evidence not in evidences:
                        evidences.append(evidence)
                else:
                    break
    else:
        while len(evidences) < 5:
            for page in pages_by_confidence:
                page = get_canonical_title(page)
                idx = page_answers[page]
                for element in idx:
                    if len(evidences) < 5:
                        element_type, element_id = element.split("_", 1)
                        evidence = [page, element_type, element_id]
                        if evidence not in evidences:
                            evidences.append(evidence)
                            break
                        else:
                            continue
                    else:
                        break
        
    
    evidences.extend(cell_evidences) # Cell evidences are included
    
    output = {"predicted_label": predicted_label, "predicted_evidence": evidences}
    
    with open(output_path, 'a', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=None)
        f.write("\n")
        
    
    if EVAL and data["label"] and data["evidence"]:

        evaluation = {"claim": data["claim"], "label": data["label"], "predicted_label": predicted_label, 
                      "evidence": data["evidence"], "predicted_evidence": []}
        
        for evidence in evidences:
            evidence_str = evidence[0] + "_" + evidence[1] + "_" + evidence[2]
            evaluation["predicted_evidence"].append(evidence_str)

        with open(evaluation_path, 'a', encoding='utf-8') as f:
            json.dump(evaluation, f, ensure_ascii=False, indent=None)
            f.write("\n")

#     print(claim)
#     print(output)

In [64]:
from tqdm import tqdm
skipped_row = 3500+2302
for data in tqdm(list(dataset[skipped_row-1:skipped_row])):
    print("Line in the original file:",skipped_row)
    
    claim = data["claim"]
    
    if not claim:
        continue
        
#     print(claim)
    
    hit_stats, hit_dictionary = get_relevant_pages(claim, N_RESULTS=10)
    
#     hit_dictionary
#     break
    
    page_answers, full_abs_list, ranked_aswers, pandasData = get_answers(claim, hit_dictionary, display_table=False)
#     print(page_answers)

#     print(pandasData)
#     continue

    entities = get_entities(claim)
    people = get_people(claim)

    max_cell_evidence = 25
    
    cell_evidences = []
    for row in pandasData:
        if row[2] < 0.5:
            continue
            
        page = row[3]
        if len(cell_evidences) >= max_cell_evidence:
            break
            
        cells = get_relevant_cells(page, [entity for entity in entities if entity not in people])
        
        model_cells = get_table_evidence(page, claim)
        
#         if not cells and not model_cells:
#             continue
            
#         if not cells:
#             continue
        
        for cell in model_cells:
            if cell not in cells:
                cells.append(cell)
        
        cells = cells[0:min([len(cells),(max_cell_evidence-len(cell_evidences))])]
        
        
        for cell in cells:
            cell_evidences.append([page, cell.split("_", 1)[0], cell.split("_", 1)[1]])
        
#         cell_evidences.extend(cells[0:min([len(cells),(max_cell_evidence-len(cell_evidences))])])
        
    cell_evidences = cell_evidences[0:max_cell_evidence] # Just to be sure
#     print(cell_evidences)
    
    
    
#     print(cell_evidences)

    evidence_probs, verdict = CheckEntailmentNeturalorContradict(ranked_aswers,claim,pandasData)

    if verdict == "False":
        predicted_label = "REFUTES"
    elif verdict == "True":
        predicted_label = "SUPPORTS"
    else:
        predicted_label = "NOT ENOUGH INFO"
        
#     entities = get_entities(claim)
    people = get_people(claim)
    
    for row_id, row in enumerate(pandasData):
        title = row[3]
        for person in people:
            if title in person or person in title:
                pandasData[row_id][2] = pandasData[row_id][2] * 2
                break
    
    pages_by_confidence = [row[0] for row in pandasData]
    
    
#     print(pandasData[0][2])
#     print(pages_by_confidence)
#     print(page_answers["Abraham Annan"])
    
    evidences = []
    
    if EVIDENCE_RETRIEVAL == "depth":
        for page in pages_by_confidence:
            page = get_canonical_title(page)
            idx = page_answers[page]
            for element in idx:
                if len(evidences) < 5:
                    element_type, element_id = element.split("_", 1)
                    evidence = [page, element_type, element_id]
#                     if len(evidences) == 0:
#                         print(evidence)
                    if evidence not in evidences:
                        evidences.append(evidence)
                else:
                    break
    else:
        while len(evidences) < 5:
            for page in pages_by_confidence:
                page = get_canonical_title(page)
                idx = page_answers[page]
                for element in idx:
                    if len(evidences) < 5:
                        element_type, element_id = element.split("_", 1)
                        evidence = [page, element_type, element_id]
                        if evidence not in evidences:
                            evidences.append(evidence)
                            break
                        else:
                            continue
                    else:
                        break
        
    
    evidences.extend(cell_evidences) # Cell evidences are included
    
    output = {"predicted_label": predicted_label, "predicted_evidence": evidences}
    print(output)
    
    with open("error_manual.jsonl", 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=None)
        f.write("\n")

#     print(claim)
#     print(output)

  0%|          | 0/1 [00:00<?, ?it/s]

Line in the original file: 5802


100%|██████████| 1/1 [00:04<00:00,  4.33s/it]

{'predicted_label': 'NOT ENOUGH INFO', 'predicted_evidence': [['Emblem of Andalusia', 'sentence', '0'], ['Genalguacil', 'sentence', '0'], ['Genalguacil', 'cell', '0_1_1'], ['Genalguacil', 'header', 'cell_0_3_0'], ['Genalguacil', 'header', 'cell_0_1_0']]}


In [24]:
searcher.search("The number three is \"Gurba(n)\" in Classical Mongolian, \"Gurav\" in Khalka, and \"Gurb\" in Buryat.", k=10)[0].docid

'Buryat language'

In [19]:
with open("test_3500_reverse.jsonl", "r") as read_file:
    lines = read_file.readlines()
    
with open("test_3500_reordered.jsonl", 'w', encoding='utf-8') as f:
    for line in reversed(lines):
        f.write(line)

In [14]:
# sql_session.execute("select * from wiki where id = 'Vítor Oliveira'").fetchone()

# json.loads(sql_session.execute("select * from wiki where id = 'John Laurie'").fetchone()[1])["table_1"]

In [15]:
# get_table_evidence("John Laurie", "In John Laurie's partial television credits, he was part of the Bees on the Boat-Deck in 1939 and Jackanory in 1971.")
                    
#         display(table)
#         print("")
#         for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
#            print(query)
#            if predicted_agg == "NONE":
#              print("Predicted answer: " + answer)
#            else:
#              print("Predicted answer: " + predicted_agg + " > " + answer)

In [16]:
# list_table = table_to_2d(BeautifulSoup(get_html_table(json.loads(sql_session.execute("select * from wiki where id = 'John Laurie'").fetchone()[1])["table_1"]["table"])))

# dict_table = {}

# for row_id, row in enumerate(list_table):
#     if row_id == 0:
#         for col_id, column in enumerate(row):
#             dict_table[column] = [row[col_id] for row in list_table[1:]]
#     break
    
# dict_table